In [ ]:
# 우리 도메인의 특성을 고려하여 필터링 과정을 RDB에 위임한 상황이다.
# 따라서 사용자 질문에서 키워드를 추출하고, 이를 SQL로 변환하여 동적 쿼리를 생성해야 한다.
# 방향성은 크게 두 가지인 것 같다.
# 1. 사용자 질의에서 데이터 추출(json 등)은 LLM이 하고, SQL 쿼리 변환은 코드가 담당하는 것,.
# 2. 데이터 추출부터 SQL 쿼리 변환까지 모두 LLM이 담당하는 것.
# 둘 중 무엇을 할까 하다가 1번을 하기로 하였다.
# 1번의 경우 유연성은 다소 떨어질 수 있지만, 안정성과 보안이 뛰어나며, 예측 및 제어 가능하다는 장점이 있기 때문이다.
# 2번은 반대로 유언성이 높지만 SQL injection 등 보안 공격에 취약하며 제어나 디버깅이 어렵다는 단점이 있기 때문이다.

# 가보자.
# 1. 사용자 질문이 들어온다 
# 2. LLM이 키워드를 추출한다(대신 이전에 비해 필터 수가 상당히 감소함. 나이, 거주지, 현재 신청 가능여부 정도만 필터링함). 
# 3. 이를 기반으로 코드 단에서 동적으로 SQL 쿼리를 생성하여 조건에 부함하는 정책 id를 가져온다.


In [ ]:
# 1. 파서 함수 : 사용자의 질문을 받아 JSON 스키마에 맞춰 필터를 생성하는 함수.

import os
import json
from openai import OpenAI

def create_filter_from_query(client: OpenAI, user_query: str) -> dict:
    """
    사용자의 자연어 질문을 분석하고,
    정책 필터링에 사용할 구조화된 JSON(파이썬 딕셔너리)을 생성합니다.

    Args:
        client: 초기화된 OpenAI 클라이언트 객체
        user_query: 사용자의 원본 질문 문자열

    Returns:
        추출된 필터 정보가 담긴 딕셔너리
    """
    # LLM에게 전달할 시스템 프롬프트. 역할, 지시사항, 허용 값, JSON 스키마를 명시합니다.
    system_prompt = """
# ROLE
You are an expert at extracting key information for filtering South Korean youth policies from a user's query.

# INSTRUCTION
- Analyze the user's query and generate a JSON object that strictly follows the provided `JSON SCHEMA`.
- CRITICAL: When extracting regions, you MUST normalize them to their full official administrative names. (e.g., "서울", "서울시" -> "서울특별시" / "경기" -> "경기도" / "부산" -> "부산광역시" / "성남" -> "성남시" / "종로" -> "종로구")
- For fields with `ALLOWED VALUES`, you MUST choose from the provided list. If a user's term is a synonym, map it to the correct value (e.g., "실업자" -> "미취업").
- If a value is not mentioned, use `null` for single values or an empty list `[]` for array values.
- Do NOT make up values that are not in the `ALLOWED VALUES` list.
- Output ONLY the JSON object.

# ALLOWED VALUES
- 'job_status': ["재직자", "자영업자", "미취업자", "프리랜서", "일용근로자",
"(예비)창업자", "단기근로자", "영농종사자", "기타", "제한없음"]
- 'marriage_status': ["기혼", "미혼", "제한없음"]
- 'education_levels': ["고졸 미만", "고교 재학", "고졸 예정", "고교 졸업", "대학 재학", "대졸 예정",
"대학 졸업", "석·박사", "기타", "제한없음"]
- 'majors': ["인문계열", "사회계열", "상경계열", "이학계열", "공학계열", 
"예체능계열", "농산업계열", "기타", "제한없음"]
- 'categories': ["일자리", "주거", "교육", "복지문화", "참여권리"]
- 'subcategories': ["취업", "재직자", "창업", "주택 및 거주지", "기숙사",
"전월세 및 주거급여 지원", "미래역량강화", "교육비지원", "온라인교육", "취약계층 및 금융지원",
"건강", "예술인지원", "문화활동", "청년참여", "정책인프라구축", "청년국제교류", "권익보호"]
- 'specializations': ["중소기업", "여성", "기초생활수급자", "한부모가정", "장애인",
"농업인", "군인", "지역인재", "기타", "제한없음"]
- 'keywords': ["대출", "보조금", "바우처", "금리혜택", "교육지원", "맞춤형상담서비스",
"인턴", "벤처", "중소기업", "청년가장", "장기미취업청년", "공공임대주택",
"신용회복", "육아", "출산", "해외진출", "주거지원"]


# JSON SCHEMA
{
  "age": "number | null",
  "income": "number | null",
  "regions": ["string"],
  "job_status": ["string"],
  "marriage_status": "string | null",
  "education_levels": ["string"],
  "majors": ["string"],
  "categories": ["string"],
  "subcategories": ["string"],
  "specializations": ["string"],
  "keywords": ["string"]
}

# EXAMPLES
---
user_query: "서울 사는 25세 미취업자인데, 창업 지원금 좀 알아봐줘"
{
  "age": 25,
  "income": null,
  "regions": ["서울특별시"],
  "job_status": ["미취업자", "(예비)창업자"],
  "marriage_status": null,
  "education_levels": [],
  "majors": [],
  "categories": ["일자리"],
  "subcategories": ["창업"],
  "specializations": [],
  "keywords": ["보조금", "벤처"]
}
---
user_query: "강원 춘천에 거주하는 고졸 학력으로 지원 가능한 주거 대출 정책 있어?"
{
  "age": null,
  "income": null,
  "regions": ["강원특별자치도", "춘천시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["고교 졸업"],
  "majors": [],
  "categories": ["주거"],
  "subcategories": ["주택 및 거주지", "기숙사", "전월세 및 주거급여 지원"],
  "specializations": [],
  "keywords": ["대출"]
}
---
user_query: "목포에 사는 사람인데 석사 지원 정책같은거 있냐"
{
  "age": null,
  "income": null,
  "regions": ["전라남도", "목포시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["석·박사"],
  "majors": [],
  "categories": [],
  "subcategories": [],
  "specializations": [],
  "keywords": []
}

---
user_query: "전국 단위로 지원해주는 청년 창업 정책 알려줘"
{
  "age": null,
  "income": null,
  "regions": [],
  "job_status": ["(예비)창업자"],
  "marriage_status": null,
  "education_levels": [],
  "majors": [],
  "categories": ["일자리"],
  "subcategories": ["창업"],
  "specializations": [],
  "keywords": []
}
"""

    try:
        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o", 
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )
        
        # 반환된 JSON 문자열을 파이썬 딕셔너리로 파싱
        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

In [ ]:
from dotenv import load_dotenv
load_dotenv()

os.getenv('OPENAI_API_KEY')
client = OpenAI()

# 사용자 질문
sample_query = "화순 사는 쌉백수인데, 관련 정책 좀 알아봐줘"

# 함수 호출
extracted_filters = create_filter_from_query(client, sample_query)

# 결과 출력
import pprint
pprint.pprint(extracted_filters)

In [ ]:
import mysql.connector
from mysql.connector import Error
def _get_all_related_region_codes(cursor, region_names: list) -> list:
    """
    지역명 리스트를 받아 관련된 모든 지역 코드(시/도 및 하위 시/군/구)를 반환합니다.
    """
    if not region_names:
        return []
    all_codes = set()
    region_regex = '|'.join(region_names)
    
    query = "SELECT code FROM region_codes WHERE sido REGEXP %s OR sigungu REGEXP %s"
    cursor.execute(query, (region_regex, region_regex))
    results = cursor.fetchall()
    for row in results:
        all_codes.add(row[0])
    
    return list(all_codes)

def get_rdb_candidate_ids(db_connection, filters: dict) -> list:
    """
    [최소 조건 버전] 기간과 지역 필터만을 사용하여 RDB에서 1차 후보군을 조회합니다.
    """
    try:
        cursor = db_connection.cursor()

        base_query = "SELECT DISTINCT p.policy_id FROM policies p"
        joins = set()
        where_conditions = []
        params = []

        # 1. 신청 기간 필터
        application_date_condition = """
        (p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))
        """
        where_conditions.append(application_date_condition.strip())
        
        # 2. 사업 기간 필터
        biz_date_condition = "(p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))"
        where_conditions.append(biz_date_condition)

        # 3. 지역 필터
        if filters.get("regions"):
            region_codes = _get_all_related_region_codes(cursor, filters["regions"])
            if region_codes:
                joins.add("JOIN policy_regions pr ON p.policy_id = pr.policy_id")
                placeholders = ', '.join(['%s'] * len(region_codes))
                where_conditions.append(f"pr.region_code IN ({placeholders})")
                params.extend(region_codes)
        
        # 최종 쿼리 조립
        final_query = base_query
        if joins:
            final_query += " " + " ".join(list(joins))
        if where_conditions:
            final_query += " WHERE (" + ") AND (".join(where_conditions) + ")"
        final_query += ";"

        # 디버깅을 위해 쿼리 템플릿과 파라미터를 별도로 출력
        print("--- Generated SQL Query (Simplified) ---")
        print(final_query)
        print("\n--- SQL Parameters ---")
        print(params)

        # 쿼리 실행
        cursor.execute(final_query, params)
        candidate_ids = [item[0] for item in cursor.fetchall()]
        return candidate_ids

    except Error as e:
        print(f"Database error: {e}")
        return []
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()

In [ ]:
# 0728 로직 개선 버전

import mysql.connector
from mysql.connector import Error

def get_rdb_candidate_ids(db_connection, filters: dict) -> list:
    """
    [최종 완성 버전] 기간, '전국 단위 정책'과 '사용자 지역 정책'을 모두 포함하여 후보군을 조회합니다.
    - 실제 DB 스키마('operation_inst')를 반영하고, 전국 단위 기관 리스트를 보강했습니다.
    """
    try:
        cursor = db_connection.cursor()

        base_query = "SELECT p.policy_id FROM policies p"
        where_conditions = []
        params = []
        
        # --- 날짜 필터 (기존과 동일) ---
        date_conditions = []
        date_conditions.append("""
        (p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))
        """)
        date_conditions.append("(p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))")
        where_conditions.append(f"({' AND '.join(date_conditions)})")


        # --- 지역 필터 로직 ---
        # 👈 중요: 실제 운영 시, 국가 부처 리스트는 DB에서 관리하거나 설정 파일로 분리하는 것이 좋습니다.
        national_institutions = [
            # 부처
            '고용노동부', '여성가족부', '행정안전부', '보건복지부', '국토교통부', 
            '중소벤처기업부', '교육부', '과학기술정보통신부', '외교부', '통일부',
            '법무부', '국방부', '문화체육관광부', '농림축산식품부', '산업통상자원부',
            '해양수산부', '환경부', 
            # 처
            '국가보훈처', '인사혁신처', '법제처', '식품의약품안전처',
            # 청
            '국세청', '관세청', '조달청', '통계청', '검찰청', '병무청', '방위사업청',
            '경찰청', '소방청', '문화재청', '농촌진흥청', '산림청', '특허청', '기상청',
            '해양경찰청',
            # 주요 공공기관
            '한국장학재단', '디지털정부', '근로복지공단', '국민연금공단', 
            '국민건강보험공단', '한국산업인력공단', '한국토지주택공사'
        ]
        
        region_filter_parts = []
        
        # 1. 전국 단위 정책 포함 조건
        placeholders = ', '.join(['%s'] * len(national_institutions))
        # ⭐ 컬럼명을 `p.operation_inst`로 수정
        region_filter_parts.append(f"p.operation_inst IN ({placeholders})")
        params.extend(national_institutions)

        # 2. 지역 단위 정책 포함 조건
        if filters.get("regions"):
            region_regex = '|'.join(filters["regions"])
            # ⭐ 컬럼명을 `p.operation_inst`로 수정
            region_filter_parts.append("p.operation_inst REGEXP %s")
            params.append(region_regex)
            
        # 1번과 2번을 OR로 연결
        if region_filter_parts:
            where_conditions.append(f"({' OR '.join(region_filter_parts)})")

        # --- 최종 쿼리 조립 ---
        final_query = base_query
        if where_conditions:
            final_query += " WHERE " + " AND ".join(where_conditions)
        final_query += ";"

        print("--- Generated SQL Query (Simplified) ---")
        print(final_query)
        print("\n--- SQL Parameters ---")
        print(params)

        cursor.execute(final_query, params)
        candidate_ids = [item[0] for item in cursor.fetchall()]
        return candidate_ids

    except Error as e:
        print(f"Database error: {e}")
        return []
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()

In [ ]:
def create_db_connection(host_name, user_name, user_password, db_name):
    """
    MySQL 데이터베이스 연결을 생성합니다.
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"MySQL 서버 버전: {db_info}")
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print(f"연결된 데이터베이스: {record}")
            return connection
    except Error as e:
        print(f"데이터베이스 연결 오류 발생: {e}")
    return connection

In [ ]:
conn = create_db_connection("localhost", "root", "1234", "toyprj4")

In [ ]:
d = get_rdb_candidate_ids(conn, extracted_filters)
pprint.pprint(d)

# %s : SQL 인젝션 방지 목적의 플레이스 홀더.

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

def semantic_search(
    candidate_ids: list,
    original_query: str,
    extracted_filters: dict,
    vdb_directory: str,
    k: int = 5,
    fetch_k: int = 20,
    lambda_mult: float = 1.0
) -> list:
    '''
    전달받은 필터를 통해 사용자의 질문을 증강하고, 후보 인덱스 내에서만 R을 수행하여 검색 정확도를 높여 시멘틱 서칭을 수행하는 함수입니다.
    Args:
        candidate_ids (list): 검색할 후보 ID 목록입니다.
        original_query (str): 원본 검색 쿼리 문자열입니다.
        extracted_filters (dict): 추출된 필터 정보 딕셔너리입니다.
        vdb_directory (str): 벡터 데이터베이스 디렉토리 경로입니다.
        k (int, optional): 반환할 상위 검색 결과의 개수입니다. 기본값은 5입니다.
        fetch_k (int, optional): 유사도 검색을 위해 가져올 초기 결과의 개수입니다. 기본값은 20입니다.
        lambda_mult (float, optional): 재정렬(reranking) 시 사용되는 람다 값입니다. 기본값은 0.7입니다.
    '''
    if not candidate_ids:
        return []

    # 1. 보강된 검색어 생성
    boost_keywords = []
    # soft_filter_keys = ["job_status", "education_levels", "keywords", "categories"]
    soft_filter_keys = []
    for key in soft_filter_keys:
        if extracted_filters.get(key):
            boost_keywords.extend(extracted_filters[key])
    
    synthetic_query = original_query + " " + " ".join(list(set(boost_keywords)))
    print(f"\n--- Generated Vector Search Query ---\n{synthetic_query}\n")

    # 2. 임베딩 모델 및 ChromaDB 로드
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = Chroma(
        collection_name="policy_collection_summary_added_openai_large", 
        
        embedding_function=embedding_model,
        persist_directory=vdb_directory
    )

    # 3. 필터가 적용된 Retriever 생성
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": k,
            "fetch_k": fetch_k,
            "lambda_mult": lambda_mult,
            "filter": {'plcyNo': {'$in': candidate_ids}}
        }
    )

    # 4. Retriever 실행 및 Document 리스트 반환
    docs = retriever.invoke(synthetic_query)
    
    return docs

In [ ]:
docs = semantic_search(d,sample_query,extracted_filters,'chroma_db_policy')
# docs = semantic_search(d,sample_query,extracted_filters,'chroma_db_policy_chunk_kss_model_tiny/')
print(docs)

In [ ]:
# 필요한 모든 함수와 라이브러리를 임포트했다고 가정합니다.
# from step1_parser import create_filter_from_query
# from step2_rdb_filter import get_rdb_candidate_ids
# from step3_semantic_search import semantic_search
from openai import OpenAI
import mysql.connector

# --- 0. 기본 설정 ---
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
VDB_DIRECTORY = "./chroma_db_policy"
openai_client = OpenAI()

# --- 1. 사용자 질문 ---
# user_query = "구로구에 사는데 주거지원 정책좀 알려줘."
# user_query = "대구 웹툰 캠퍼스"
user_query = "웹툰캠퍼스 운영 및 인력 양성 정책에 대해 알려줘."
# user_query = "웹툰 작가가 되고 싶은데, 교육이나 지원금 받을 수 있는 프로그램 찾아줘."
# user_query = "벤처 지원이랑 맞춤형 상담 서비스를 동시에 해주는 문화 정책이 궁금해."

print(f"--- User Query ---\n{user_query}\n")


# --- 2. 1단계: LLM 파서 실행 ---
llm_filters = create_filter_from_query(openai_client, user_query)
print(f"--- Step 1: LLM Filter Results ---\n{llm_filters}\n")
# --- 3. 2단계: RDB 필터링 실행 ---
try:
    connection = mysql.connector.connect(
        host='localhost',
        database='toyprj4',
        user='root',
        password='1234'
    )
    rdb_candidate_ids = get_rdb_candidate_ids(connection, llm_filters)
    print(f"--- Step 2: RDB Candidate IDs ---\n{rdb_candidate_ids}\n")

finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()


# --- 4. 3단계: Vector DB 랭킹 실행 ---
final_documents = semantic_search(
    candidate_ids=rdb_candidate_ids,
    original_query=user_query,
    extracted_filters=llm_filters,
    vdb_directory=VDB_DIRECTORY
)


# --- 5. 최종 결과 확인 ---
print("\n--- Step 3: Final Retrieved Documents ---")
for doc in final_documents:
    print(f"Policy ID: {doc.metadata.get('plcyNo')}")
    print(f"application date: {doc.metadata.get('aplyYmd')}")
    print(f"Content: {doc.page_content}...") # 내용 일부만 출력
    print("-" * 20)

In [ ]:
# 오케이 일단 확인 완료

In [ ]:
import chromadb

# 본인의 Vector DB 경로를 정확하게 입력해주세요.
VDB_DIRECTORY = "./chroma_db_policy/" 
# 인덱싱 시 사용한 컬렉션 이름을 정확하게 입력해주세요.
COLLECTION_NAME = "policy_collection_summary_added_openai_large" 

try:
    # 1. 저장된 DB 로드
    client = chromadb.PersistentClient(path=VDB_DIRECTORY)
    collection = client.get_collection(name=COLLECTION_NAME)

    # 2. 필터 없이 데이터 5개 가져오기
    results = collection.get(
        limit=1,
        include=["metadatas", "documents"] # 메타데이터와 문서 내용을 함께 확인
    )

    # 3. 결과 출력
    print("--- ChromaDB에 저장된 데이터 샘플 ---")
    import pprint
    pprint.pprint(results)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")
    print("Vector DB 경로와 컬렉션 이름이 올바른지 다시 확인해주세요.")